### Browse Quotes Inbound
Retrieve the cheapest quotes from our cache prices. 

In [1]:
import requests

url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/UA/USD/en-DB/LWO-sky/TFS-sky/2021-10-22/2021-10-30"

headers = {
    'x-rapidapi-key': "58bd283fc5msh32f4f5e967b9667p12f05fjsn8548086bfcf3",
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers)

print(response.text)

{
  "Quotes" : [ ],
  "Carriers" : [ ],
  "Places" : [ ],
  "Currencies" : [ {
    "Code" : "USD",
    "Symbol" : "$",
    "ThousandsSeparator" : ",",
    "DecimalSeparator" : ".",
    "SymbolOnLeft" : true,
    "SpaceBetweenAmountAndSymbol" : false,
    "RoundingCoefficient" : 0,
    "DecimalDigits" : 2
  } ]
}


### Brouse Dates
Retrieve the cheapest dates for a given route from our cache.

In [2]:
import requests

url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsedates/v1.0/UA/UAH/ukr-UA/LWO-sky/TFS-sky/anytime"

querystring = {"inboundpartialdate":"2019-12-01"}

headers = {
    'x-rapidapi-key': "58bd283fc5msh32f4f5e967b9667p12f05fjsn8548086bfcf3",
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{
  "Quotes" : [ {
    "QuoteId" : 1,
    "MinPrice" : 680,
    "Direct" : false,
    "OutboundLeg" : {
      "CarrierIds" : [ 1090 ],
      "OriginId" : 66415,
      "DestinationId" : 84035,
      "DepartureDate" : "2021-11-08T00:00:00"
    },
    "QuoteDateTime" : "2021-10-22T09:22:00"
  }, {
    "QuoteId" : 2,
    "MinPrice" : 888,
    "Direct" : false,
    "OutboundLeg" : {
      "CarrierIds" : [ 1090 ],
      "OriginId" : 66415,
      "DestinationId" : 84035,
      "DepartureDate" : "2021-12-06T00:00:00"
    },
    "QuoteDateTime" : "2021-10-20T06:52:00"
  }, {
    "QuoteId" : 3,
    "MinPrice" : 1598,
    "Direct" : false,
    "OutboundLeg" : {
      "CarrierIds" : [ 1878 ],
      "OriginId" : 66415,
      "DestinationId" : 84035,
      "DepartureDate" : "2022-01-19T00:00:00"
    },
    "QuoteDateTime" : "2021-10-19T10:50:00"
  }, {
    "QuoteId" : 4,
    "MinPrice" : 1623,
    "Direct" : false,
    "OutboundLeg" : {
      "CarrierIds" : [ 1090 ],
      "OriginId" : 66415,
      

In [3]:
def mapping_carriers(data):
    # [{'CarrierId': 1878, 'Name': 'Wizz Air'}, {'CarrierId': 1090, 'Name': 'Ryanair'}]  -> {1878: 'Wizz Air', 1090: 'Ryanair'}
    return {name_id['CarrierId']: name_id['Name'] for name_id in data}


def mapping_places(data):
    # {'Name': 'Lviv', 'Type': 'Station', 'PlaceId': 66415, 'IataCode': 'LWO', 'SkyscannerCode': 'LWO',
    #  'CityName': 'Lviv', 'CityId': 'LVOV', 'CountryName': 'Ukraine'}
    #     ->
    # 66415 : ['Lviv', 'Ukraine']
    return {name_id['PlaceId']: [name_id['CountryName'], name_id['Name'], name_id['IataCode']] for name_id in data}

def result_for_one_response(result_d):
    data = result_d.copy()
    data['OutboundLeg']['CarrierIds'] = CARRIERS[data['OutboundLeg']['CarrierIds'][0]]
    data['OutboundLeg']['OriginId'] = PLACES[data['OutboundLeg']['OriginId']]
    data['OutboundLeg']['DestinationId'] = PLACES[data['OutboundLeg']['DestinationId']]
    return data


In [4]:
def normalize_data(input_data):
    return f"""
        {input_data['QuoteId']}
        Company name: {input_data['OutboundLeg']['CarrierIds']}
        From:
        Country: {input_data['OutboundLeg']['OriginId'][0]}
        Code: {input_data['OutboundLeg']['OriginId'][2]}
        City: {input_data['OutboundLeg']['OriginId'][1]}
        Date: {input_data['QuoteDateTime']}
        To:\n
        Country: {input_data['OutboundLeg']['DestinationId'][0]}
        Code: {input_data['OutboundLeg']['DestinationId'][2]}
        City: {input_data['OutboundLeg']['DestinationId'][1]}
        Date: {input_data['OutboundLeg']['DepartureDate']}
        Price: {input_data['MinPrice']}"""


QUOTES = response.json().get('Quotes')
PLACES = mapping_places(response.json().get('Places'))
CURRENCIES = response.json().get('Currencies')
DATES = response.json().get('Dates')
CARRIERS = mapping_carriers(response.json().get('Carriers'))



for result in QUOTES:
    result_data = result_for_one_response(result)
    print(normalize_data(result_data))
    print('_____<<<>>>______')





        1
        Company name: Ryanair
        From:
        Country: Ukraine
        Code: LWO
        City: Lviv
        Date: 2021-10-22T09:22:00
        To:

        Country: Spain
        Code: TFS
        City: Tenerife South
        Date: 2021-11-08T00:00:00
        Price: 680
_____<<<>>>______

        2
        Company name: Ryanair
        From:
        Country: Ukraine
        Code: LWO
        City: Lviv
        Date: 2021-10-20T06:52:00
        To:

        Country: Spain
        Code: TFS
        City: Tenerife South
        Date: 2021-12-06T00:00:00
        Price: 888
_____<<<>>>______

        3
        Company name: Wizz Air
        From:
        Country: Ukraine
        Code: LWO
        City: Lviv
        Date: 2021-10-19T10:50:00
        To:

        Country: Spain
        Code: TFS
        City: Tenerife South
        Date: 2022-01-19T00:00:00
        Price: 1598
_____<<<>>>______

        4
        Company name: Ryanair
        From:
        Country: Ukraine
   

In [2]:
import pandas as pd
data = 'C:/Users/Kate/Downloads/airports_iata_codes.csv'
airport_code = pd.read_csv(data)
#airport_code.head()
airport_code[[name, iata_code]].head()


NameError: name 'name' is not defined

In [8]:
from requests import request

BASE_URL = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices"
#UA/UAH/ukr-UA/LWO-sky/TFS-sky/anytime

headers = {
    'x-rapidapi-key': "58bd283fc5msh32f4f5e967b9667p12f05fjsn8548086bfcf3",
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com"
    }


def quotes(country, currency, locale, origin, destination, outbound_partial_date, inbound_partial_date=None):
    return request("GET",
                   f"{BASE_URL}/browsedates/v1.0/{country}/{currency}/{locale}/{origin}/{destination}/{outbound_partial_date}",
                  headers=headers)

#res = quotes("UA", "UAH", "ukr-UA", "LWO-sky", "TFS-sky", "anytime").json()
res = quotes("UA", "UAH", "ukr-UA", "LWO-sky", "CDG-sky", "anytime").json()

In [7]:
from pprint import pprint

pprint(res)

{'Carriers': [{'CarrierId': 1878, 'Name': 'Wizz Air'},
              {'CarrierId': 1755, 'Name': 'Turkish Airlines'},
              {'CarrierId': 1090, 'Name': 'Ryanair'},
              {'CarrierId': 1375, 'Name': 'LOT'}],
 'Currencies': [{'Code': 'UAH',
                 'DecimalDigits': 2,
                 'DecimalSeparator': ',',
                 'RoundingCoefficient': 0,
                 'SpaceBetweenAmountAndSymbol': False,
                 'Symbol': 'грн.',
                 'SymbolOnLeft': False,
                 'ThousandsSeparator': '\xa0'}],
 'Dates': {'OutboundDates': [{'PartialDate': '2021-10',
                              'Price': 2742,
                              'QuoteDateTime': '2021-10-22T06:08:00',
                              'QuoteIds': [9]},
                             {'PartialDate': '2021-11',
                              'Price': 1186,
                              'QuoteDateTime': '2021-10-22T08:27:00',
                              'QuoteIds': [1]},
      